# Drugi etap projektu
Julia Jodczyk

Filip Pawłowski 
### Polecenie:
“Jakiś czas temu wprowadziliśmy konta premium, które uwalniają użytkowników od słuchania reklam. Nie są one jednak jeszcze zbyt popularne – czy możemy się dowiedzieć, które osoby są bardziej skłonne do zakupu takiego konta?”

In [4]:
import pickle
import requests
import json
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

from microservice.load_data import Preprocessor, DataModel
from microservice.files_utils import randomly_split_group

## Modele

Stworzyliśmy modele klasyfikacji binarnej, które dzielą użytkowników na grupy: `kupi premium` i `nie kupi premium`. 

### Model Bazowy

Jako model bazowy, najprostszy z możliwych dla danego zadania uznajemy model naiwny, który zawsze klasyfikuje użytkowników do grupy `kupi premium`: 

In [5]:
class NaiveModel:
    def predict(self, input_df):
        user_ids = input_df.index
        mock_series = pd.Series(True, index=user_ids, name="user_id")
        return mock_series
    
base_model = NaiveModel()

with open('./microservice/saved_models/base_model.sav', 'wb') as f:
    pickle.dump(base_model, f)

### Model docelowy

Wybierając model docelowy przeprowadziliśmy serię przeszukiwań RandomizedSearch, w których ocenialiśmy sprawność modeli przy użyciu K-fold cross walidacji. Początkowo porównywane były klasifikatory: K-najbliższych sąsiadów oraz minimalno-odległościowy. Na dalszych etapach projektu dodaliśmy do tego zbioru również inne modele - np. RandomForest oraz DecisionTree. Osiągały one bardzo dobre oraz podobne wyniki do wcześniej wytypowanego klasyfikatora K-najliższych sąsiadów, w związku z czym w realnym projekcie prawdopodobnie rozważylibyśmy dostarczenie np. dwóch pretrenowanych modeli i pozwolenie klientowi na wybór w czasie rzeczywistym trwania programu jednego z nich i porównywanie osiąganych efektów. 
W naszym wypadku dalsze rozważania zawrzemy w kontekście jednego modelu - KNeighboursClassifier. Po generacji nowych atrybutów z danych, ostatecznymi cechami (per użytkownik) dla klasyfikatora są:
- miasto 
- stosunek czasu reklam do całego czasu, jaki użytkownik spędził korzystając z serwisu
- stosunkowy udział każdego typu zdarzenia (event_type) we wszystkich zdarzeniach sesji
- stosunek ilości reklam po utworach ulubionego gatunku
- ulubione gatunki użytkownika

Implementacja ekstrakcji powyższych cech została umieszczona w pliku `load_data.py`. Cechy nieliczbowe - miasto oraz ulubione gatunki zostały zakodowane sposobem one hot encoding.

Strojenie hiperparametrów zawarto w pliku `tuning.py`.

Sposób oceny modeli opisano w sekcji "Porównanie wyników offline"   
 

In [6]:
target_model = KNeighborsClassifier()
# load data:
data_model = DataModel()
data_model.users_df = pd.read_json("./data/users.json")
df = data_model.get_merged_dfs()
preprocessed_df = Preprocessor.transform(df)

In [7]:
preprocessed_df

,premium_user,Gdynia,Kraków,Poznań,Radom,Szczecin,Warszawa,Wrocław,Ads_ratio,adds_after_fav_ratio,...,ranchera,regional mexican,rock,rock en espanol,roots rock,singer-songwriter,soft rock,soul,tropical,vocal jazz
0,True,0,0,0,0,0,0,1,0.030566,0.035714,...,0,0,0,0,0,0,0,0,0,0
196,False,0,1,0,0,0,0,0,0.018199,0.223684,...,0,0,0,0,0,1,0,0,0,0
699,False,0,1,0,0,0,0,0,0.011745,0.080000,...,0,0,0,0,1,0,0,0,0,0
947,False,0,0,0,0,0,1,0,0.024229,0.000000,...,0,0,0,0,0,0,0,0,1,1
1737,False,0,0,0,0,0,1,0,0.019117,0.041096,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367214,False,0,0,0,1,0,0,0,0.031597,0.558140,...,0,0,0,0,0,0,0,0,0,0
367484,False,0,1,0,0,0,0,0,0.024226,0.094118,...,0,0,1,0,0,0,0,0,0,0
367904,True,0,0,0,0,0,0,1,0.033301,0.057143,...,0,0,0,0,0,1,0,0,0,0
368124,False,0,1,0,0,0,0,0,0.022990,0.009804,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# split data:
X, y = preprocessed_df.drop(["premium_user"], axis=1), preprocessed_df["premium_user"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=18)
target_model.fit(X_train, y_train)

with open('./microservice/saved_models/KNN_model.sav', 'wb') as f:
    pickle.dump(target_model, f)

/home/julia/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [9]:
base_y_hat = base_model.predict(X_test)
score = accuracy_score(y_test, base_y_hat)
score 

0.346031746031746

In [10]:
report = classification_report(y_test, base_y_hat)
print(report)

              precision    recall  f1-score   support

       False       0.00      0.00      0.00       206
        True       0.35      1.00      0.51       109

    accuracy                           0.35       315
   macro avg       0.17      0.50      0.26       315
weighted avg       0.12      0.35      0.18       315



/home/julia/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/julia/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/julia/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
target_y_hat = target_model.predict(X_test)
score = accuracy_score(y_test, target_y_hat)
score 

/home/julia/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


0.6571428571428571

In [12]:
report = classification_report(y_test, target_y_hat)
print(report)

              precision    recall  f1-score   support

       False       0.74      0.73      0.74       206
        True       0.50      0.51      0.51       109

    accuracy                           0.66       315
   macro avg       0.62      0.62      0.62       315
weighted avg       0.66      0.66      0.66       315



### Porównanie wyników offline

W ocenie rezultatów wykorzystano przede wszystkim dwa wyznaczniki.
Bazowym była skuteczność, jako prosta metryka dająca informację o ilości poprawnych predykcji wśród wszystkich. Jednocześnie jako wyznacznik kryterium sukcesu, była ona głównym celem dalszego rozwoju modelu. 
Ważny również był F1-score - łączy on w sobie recall oraz precision, które są kluczowe w naszym przypadku. Najważniejsze bowiem dla powodzenia klienta jest, aby jak najwięcej osób, którym polecone zostało premium, rzeczywiście było skłonnych kupić premium (innymi słowy, żeby jak najwięcej z naszych poleceń było do osób rzeczywiście zainteresowanych zakupem premium - precision) oraz abyśmy zaproponowali premium jak największej części osób skłonnych do jego zakupu - recall.

- Model bazowy, zawsze zwracający prawdę ma skuteczność na poziomie ok. 35% co zgadza się z rozkładem danych. Naszemu modelowi udało osiągnąć skuteczność na poziomie 53%. Jest to poprawa na poziomie 18 punktów procentowych. 
- Precyzja modelu docelowego również jest wyższa, zarówno dla klasy "kupi premium" jak i "nie kupi premium". Oznacza to, że nasz model jest lepszy w poprawnej identyfikacji klas,
- Model docelowy ma również lepszy wynik F1-score. Podsumowując, jest on lepszy od naiwnego we wszystkich rozpatrywanych kategoriach, poza recall, który z oczywistych względów jest 100% dla modelu naiwnego w kategorii zainteresowanych zakupem premium.

Model decelowy spełnia założone kryterium sukcesu - skuteczność na poziomie wyższym niż 35%. 

### Porównanie wyników

Wyniki predykcji zbierzemy za pomocą zaimplementowanego mikroserwisu (szczegóły implementacji i API niżej). 

(Przed uruchomieniem kodu z poniższej komórki należy uruchomić mikroserwis komendą `python3 /microservice/microservice.py`) 

In [54]:
users = pd.read_json("./data/users_new.json")
users_split = users.iloc[:200]
base_model_users, target_model_users = randomly_split_group(users_split)

In [74]:
user = preprocessed_df.iloc[[0]]
user.drop("premium_user", axis=1, inplace=True)
int(target_model.predict(user.values)[0])


/home/julia/anaconda3/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


1

In [83]:
base_url = "http://127.0.0.1:8000"

for _, user in target_model_users.iterrows():
    payload = user.to_dict()
    requests.post(f"{base_url}/predict-with/KNN", params={"test": "True"}, json=payload)
    actual_body = {
        "user_id": user["user_id"],
        "actual": user["premium_user"]
    }
    requests.post(f"{base_url}/submit-actual", json=actual_body)

for _, user in base_model_users.iterrows():
    payload = user.to_dict()
    requests.post(f"{base_url}/predict-with/base", params={"test": "True"}, json=payload)
    actual_body = {
        "user_id": user["user_id"],
        "actual": user["premium_user"]
    }
    requests.post(f"{base_url}/submit-actual", json=actual_body)


In [84]:
response = requests.get(f"{base_url}/test_ab_results")
response.json()["AB_test_verdict"]

'Fail to reject H0: No significant difference in performance between A and B'

#### Funkcjonalność Dodana w ramach poprawy etapu 2

* LabelBinarizer'y otrzymują DataFrame, który jest następnie sortowany po wartości atrybutu, który ma być zakodowany.
Dodatkowo dodano możliwość eksportu narzędzi użytych w preprocessingu, takich jak wspomniane Binarizer'y, do plików .sav. Dla takiego samego zbioru możliwych wartości, kodowanie będzie takie samo. 


In [ ]:
@staticmethod
    def save_binarizers(filenames={"mlb":"./microservice/saved_models/mlb.sav", "lb":"./microservice/saved_models/lb.sav"}):
        pickle.dump(Preprocessor.mlb, open(filenames["mlb"], "wb"))
        pickle.dump(Preprocessor.lb, open(filenames["lb"], "wb"))


* Zamieszczono funkcję ustawiającą parametry dzielone między modułami, jak logowanie i ziarno generatora losowego w funkcji config() w pliku utils.configs. Jest ona wywoływana w głównym pliku aplikacji i powoduje deterministyczne zachowanie komponentów.

In [ ]:
def config_seed(seed:int=18):
    np.random.seed(seed)
    random.seed(seed)

* Ponowne trenowanie modelu odbywa się przy pomocy ModelManagera - klasy, która agreguje model i jego metody znane z api sklearn - i.e. fit i predict. Dodatkowo posiada prepare_data, do którego można podać ścieżki do plików z danymi, podobnie do domyślnej wartości atrybutu.
* W celu korzystania w taki sposób z dostarczonych funkcjonalności, można skorzystać z kodu analogicznego do `test_model_manual.py`: 

In [ ]:
model_manager = ModelManager(KNeighborsClassifier)
# ModelManager jako domyślny model przyjmuje KNeighborsClassifier, więc nie było konieczne go podawać
model_manager.prepare_data(since=np.datetime64('2021-08', 'D')).fit_data().predict()
# Rezultaty operacji zapisywane są w obiekcie, z którego dostępne są na zewnątrz
print(model_manager.classification_report())

* Plik requiremets został dodany do katalogu microservice
* Do pierwotnej analizy dodano pozostałe uwagi, jak na przykład odniesienie się do kryterium sukcesu, które model decelowy spełnia - skuteczność na poziomie wyższym niż 35%. 
* Dodano strojenie hiperparametrów w pliku `tuning.py`